# Relational databases and SQL query basics

By [Allison Parrish](https://www.decontextualize.com/)

A relational database contains a number of tables consisting of rows and columns. Each row in the table represents some entity, with each column giving information about that entity. The tables of a relational database together are generally intended to work together to represent data in a particular domain, or for a particular system.

In this notebook, I'm going to explain the basics of how relational databases work, and give a tutorial on how to use SQL for querying information in these databases. We'll also discuss a little bit about how you can integrate Python, Pandas and SQL. This notebook does *not* contain any information on relational database design—that's a whole separate topic altogether.

As a quick example of what a relational database might look like, let’s consider the data storage needs of an imaginary news organization. A news organization might need to keep track of all the writers they have on
staff. We’ll create a “writers” table to hold this information:

| name                 | title        | start_year |
|----------------------|--------------|------------|
| Gabriella McCullough | reporter     | 2009       |
| Steven Kennedy       | drama critic | 2012       |
| Jalen Shaara         | columnist    | 2002       |

… and then a table of articles that those writers are responsible for:

| author               | title                                         | published_date |
|----------------------|-----------------------------------------------|----------------|
| Gabriella McCullough | Man, opossum reach garbage accord             | 2015-07-01     |
| Steven Kennedy       | “The Deceit of Apricot” opens to rave reviews | 2015-07-15     |
| Jalen Shaara         | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16     |
| Gabriella McCullough | Traffic signals restored on Tunguska Ave      | 2015-07-01     |

Looks pretty okay so far! You can easily imagine using this data for any number of purposes: to generate the home page of the publication; to do metric evaluations on employee performance; to do text analysis on article titles, etc.

The decisions you make about how to separate your data into tables, and how to decide what columns to put into those tables, is called the database *schema*. The schema above isn’t an okay start, but has some problems—which we’ll discuss as the tutorial progresses.

## Relational database management software (RDBMS)

The concept of the “relational database” stretches back many decades,
and over the years a number of programmers and vendors have made
available software that implements the basic idea. The most popular that
you’re likely to come across:

-   MySQL (and its fork MariaDB), an open-source RDBMS widely used in web applications
-   PostgreSQL, another open-source RDBMS
-   SQLite and DuckDB are both small embeddable RDMBS that you can include in essentially
    any application (the Python standard library includes a SQLite module) and are often used for data analysis
-   DuckDB
-   Commercial/enterprise RDBMS include: Microsoft’s SQL Server, Oracle, IBM's DB2

If you’re developing an application from scratch, it can be tricky to
decide which RDBMS to use. There are many criteria that might play into
your decision (such as: How much does it cost? How does it perform with
large amounts of data? How does it perform with a large number of
users?). (If you’re working with an existing database (say, a database
already present in your organization), you’ll just have to learn to work
with what you’re given.)

In this tutorial, we’re going to use SQLite. The primary benefits of SQLite are that it's small, easy to use, and doesn't require any extra installation steps if you already have Python installed. Although the details of SQL differ from one RDBMS to the next, almost everything we're going to discuss in this tutorial applies to any RDBMS out there.

## SQL

Although the details of how any given RDBMS stores its data can differ wildly, nearly every RDBMS you use supports one computer language for data input and data access: SQL. SQL (“Structured Query Language,” often pronounced as “sequel” and sometimes by naming its initials) is a language that is *specifically built* for specifying and retrieving combinations of rows and columns from relational data. It’s an extraordinarily powerful language, and of what follows in this tutorial is learning how SQL works and what it looks like.

SQL is essentially a standardized way to write the kinds of queries against tabular data that we've already been doing in Pandas. In fact, many Pandas functions have [direct SQL analogues](https://pandas.pydata.org/docs/getting_started/comparison/comparison_with_sql.html), and there are many ways to get [Pandas and SQL to interoperate](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html).

It's important to note that SQL is not just a query language; it's also a language for rigorously *defining* how data is structured. We'll talk more about this later, but it is something that SQL does well that Pandas lacks altogether.

In the same way that you can write HTML and have it appear basically the same way on every web browser, you can write SQL and expect it to behave more or less the same in every RDBMS. Or, at least, that’s the ideal. But beware: every RDBMS has slightly different rules for how to write SQL, and even if you’re an expert with one RDBMS, it can be non-trivial to learn how to work with another one. This tutorial shows how to use SQLite, and the concepts shown here should carry to any other system. But if you do end up using another RDBMS, be aware that you may need to consult the documentation for that RDBMS and fiddle around with the syntax.

### A quick taste

Here’s a little preview of how SQL works. Here are the tables of data that we created above for our imaginary news organization:

| name                 | title        | start_year |
|----------------------|--------------|------------|
| Gabriella McCullough | reporter     | 2009       |
| Steven Kennedy       | drama critic | 2012       |
| Jalen Shaara         | columnist    | 2002       |

… and then a table of articles that those writers are responsible for:

| author               | title                                         | published_date |
|----------------------|-----------------------------------------------|----------------|
| Gabriella McCullough | Man, opossum reach garbage accord             | 2015-07-01     |
| Steven Kennedy       | “The Deceit of Apricot” opens to rave reviews | 2015-07-15     |
| Jalen Shaara         | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16     |
| Gabriella McCullough | Traffic signals restored on Tunguska Ave      | 2015-07-01     |

The SQL commands for creating these tables in the database looks like this:

    create table reporters (name text, title text, start_year integer);

    create table articles (author text, title text, published_date date);

The SQL commands for populating those tables with data looks like this:

    insert into reporters (name, title, start_year) values
        ('Gabriella McCullough', 'reporter', 2009),
        ('Steven Kennedy', 'drama critic', 2012),
        ('Jalen Shaara', 'columnist', 2002);

    insert into articles (author, title, published_date) values
        ('Gabriella McCullough', 'Man, opossum reach garbage accord', '2015-07-01'),
        ('Steven Kennedy', '"The Deceit of Apricot" opens to rave reviews', '2015-07-15'),
        ('Jalen Shaara', 'What''s the Big Data? Why I''m a data skeptic', '2015-07-16'),
        ('Gabriella McCullough', 'Traffic signals restored on Tunguska Ave', '2015-07-01'); 

Here are some example queries we can run on the data, along with their results. To get a list just of reporter’s names:

    select name from reporters;

            name         
    ---------------------
    Gabriella McCullough
    Steven Kennedy
    Jalen Shaara

To find out how many articles a particular writer has written:

    select count(title) from articles where author = 'Gabriella McCullough';

    count 
    -------
        2

To get a list of articles and authors, along with the titles of those
authors:

    select articles.author, reporters.title, articles.title
        from articles
        join reporters on reporters.name = articles.author;

        author        |    title     |                     title                     
    ---------------------+--------------+-----------------------------------------------
    Gabriella McCullough | reporter     | Traffic signals restored on Tunguska Ave
    Gabriella McCullough | reporter     | Man, opossum reach garbage accord
    Steven Kennedy      | drama critic | "The Deceit of Apricot" opens to rave reviews
    Jalen Shaara        | columnist    | What's the Big Data? Why I'm a data skeptic

It’s like magic!

## How to interact with your database

The organizing metaphor of using an RDBMS is this: you "connect" to the "server," and then issue "commands" to the server. In many real world applications, the RDBMS will actually be running on a separate computer, and you would actually connect to that server over the network, and log in with your credentials (sort of like logging into a server over SSH, or like using a key for a web API), in which case this metaphor would be fairly literal. However, the metaphor also applies when (as in this tutorial!) you're only accessing a database stored as a file on your own computer.

Most RDBMSes have web-based interfaces or specialized text-mode programs for interaction on the command line; for SQLite, this is called the [SQLite shell](https://www.sqlite.org/cli.html), which comes installed with Python. Many tutorials will assume that you're using these command-line interfaces. However, we're going to use an interface called [JupySQL](https://jupysql.ploomber.io/en/latest/quick-start.html), which allows us to connect to an RDBMS and issue SQL commands right inside Jupyter Notebook.

To install JupySQL, run the cell below:

In [ ]:
%pip install jupysql

Now load the JupySQL extension like so:

In [21]:
%load_ext sql
%config SqlMagic.displaylimit = 50
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


(I've also set some [JupySQL configuration variables](https://jupysql.ploomber.io/en/latest/api/configuration.html) to values I like better than the defaults.)

## Your first database

For the purposes of this tutorial, I’m going to have you *import* some
data into your database. In particular, we’re going to use data from the
[MONDIAL
project](http://www.dbis.informatik.uni-goettingen.de/Mondial/), which
is a database of geographical facts gleaned from various freely
available sources. The project makes the data available in a number of SQL formats, but not SQLite; we're going to use [SQLite format files prepared by a third party](https://github.com/seralf/mondial_sqlite).

[Download this file](https://github.com/seralf/mondial_sqlite/blob/master/db/mondial.db) and put it in the same directory as this notebook.

To work with the Mondial database, we need to "connect" to it. The following cell accomplishes this:

In [22]:
%sql sqlite:///mondial.db

The `%` character at the beginning of the cell indicates a "magic" command, which is a special command that Jupyter Notebook interprets, instead of plain Python code. ([A list of all built-in magic commands can be found in the IPython documentation.](https://ipython.readthedocs.io/en/stable/interactive/magics.html)) The jupySQL package makes several additional magic commands available, including the `%sql` command used here and the `%sqlcmd` command below.

Before we actually write any SQL, I want to show a few commands you can use to see what's actually in the database. The following cell shows the names of all tables in the database:

In [23]:
%sqlcmd tables

Name
Airport
City
Citylocalname
Cityothername
Citypops
Continent
Country
Countrylocalname
Countryothername
Countrypops


... and this displays the names and types of the columns in a particular table:

In [24]:
%sqlcmd columns -t Language

name,type,nullable,default,primary_key
Country,TEXT(4),True,None,2
Name,TEXT(50),True,None,1
Percentage,NUMERIC,True,None,0


In [25]:
%sqlcmd columns -t Country

name,type,nullable,default,primary_key
Name,TEXT(50),False,None,0
Code,TEXT(4),True,None,1
Capital,TEXT(50),True,None,0
Province,TEXT(50),True,None,0
Area,NUMERIC,True,None,0
Population,NUMERIC,True,None,0


### Data types

What's a data type?

Every column in a table has a *data type*. There are a number of core
SQL data types supported by nearly every RDBMS, and individual vendors
may also supply data types specific to their own software. Some of the
most common data types in SQL are:

| type                                   | description                                         |
|----------------------------------------|-----------------------------------------------------|
| `varchar(n)` or `character varying(n)` | a string that is at most *n* characters long        |
| characters long                        |                                                     |
| `text`                                 | a string of arbitrary length                        |
| `integer` or `int`                     | integer numbers                                     |
| `float`                                | floating-point numbers                              |
| `numeric`                              | fractional numbers with a fixed precision           |
| `date`                                 | stores a year, month and day                        |
| `timestamp`                            | stores a year, month, day, hour, minute, and second |
| `boolean`                              | either `true` or `false`                            |

[Here’s a full-list of data types supported by SQLite](https://www.sqlite.org/datatype3.html).
It’s important to recognize what data types you’re working with, since
it changes the way you write queries against the data, and how some
features (like sorting) will behave.

### Writing a query with `SELECT`

Enough with the preliminaries! Let’s actually get some data out of the
database. The way you get data out of a SQL table is to write a `SELECT`
statement. The `SELECT` statement allows you to specify (among other
things):

-   which rows you want
-   for the rows selected, which columns you want
-   the number of rows to return
-   how to sort the rows

The basic syntax for `SELECT` is this:

    SELECT fields
    FROM table
    WHERE criterion
    ORDER BY order_fields
    LIMIT number;

… where:

-   `fields` is a comma-separated list of fields that you want to
    retrieve
-   `table` is the name of the table you want to query;
-   `criterion` is a SQL expression that determines with rows will be
    included (more on SQL expressions in a bit);
-   `order_fields` is a comma-separated list of fields to use as the
    basis for sorting the results (e.g., if you want to sort
    alphabetically by the `name` field, put `name` here); you can
    optionally include the `DESC` keyword here to sort in descending order.
-   `number` is the maximum number of records to return.

The `WHERE`, `ORDER BY` and `LIMIT` lines are all optional: the only
thing you need for a `SELECT` statement is a list of fields and a table.
SQL is also not white-space sensitive; I wrote each clause on a separate
line above, but you’re free to include new lines and extra whitespace
where you will. You could also potentially write the whole statement on
a single line, if you wanted to. (Just remember to end the statement
with a semicolon.)

> NOTE: This is only a small subset of the `SELECT` statement’s
> capabilities, presented for pedagogical purposes only! The actual
> syntax is [much more
> complicated](https://www.sqlite.org/lang_select.html).

So, for example, let’s select the `name` and `population` columns from
the `city` table, displaying only the rows where the value for
`population` is greater than nine million. To do this, we'll use the `%%sql` cell magic—with this magic, the entire contents of the cell will be treated as a SQL command to send to the database. The results of that command will show in the area right below.

In [26]:
%%sql
SELECT name, population
FROM city
WHERE population > 9000000;

Name,Population
Moskva,11979529
Istanbul,13710512
Guangzhou,11071424
Shenzhen,10358381
Wuhan,9785388
Beijing,11716620
Shanghai,22315474
Tianjin,11090314
Lahore,11126285
Karachi,14916456


To order these cities alphabetically, we can add an `ORDER BY` clause:

In [27]:
%%sql
SELECT name, population
FROM city
WHERE population > 9000000
ORDER BY name;

Name,Population
Beijing,11716620
Delhi,11034555
Guangzhou,11071424
Istanbul,13710512
Jakarta,9607787
Karachi,14916456
Kinshasa,11575000
Lagos,13745000
Lahore,11126285
Moskva,11979529


To order these cities by descending population:

In [28]:
%%sql
SELECT name, population
FROM city
WHERE population > 9000000
ORDER by population DESC;

Name,Population
Shanghai,22315474
Karachi,14916456
Lagos,13745000
Istanbul,13710512
Mumbai,12442373
Moskva,11979529
Beijing,11716620
Kinshasa,11575000
São Paulo,11152344
Lahore,11126285


Finally, to limit our result set to only the top five cities, we can
include the `LIMIT` keyword. Let’s change the fields to include the
country as well:

In [30]:
%%sql
SELECT name, country, population
FROM city
WHERE population > 9000000
ORDER by population DESC
LIMIT 5;

Name,Country,Population
Shanghai,CN,22315474
Karachi,PK,14916456
Lagos,WAN,13745000
Istanbul,TR,13710512
Mumbai,IND,12442373


### Exploring tables with `*` and `LIMIT`

Occasionally it’s useful to just “take a peek” at the data that’s in a
table, without having to specify which columns and rows you want in
particular. For these purposes, you can put a `*` in the field parameter
(right after the word `SELECT`), which will include *all* of the fields
in the table in your search result. Combined with the `LIMIT` clause,
you can use this to take a look at what’s in the first few rows of the
table `country`:

In [49]:
%%sql
SELECT *
FROM country
LIMIT 10;

Name,Code,Capital,Province,Area,Population
Albania,AL,Tirana,Albania,28750,2821977
Greece,GR,Athina,Attikis,131940,10816286
Macedonia,MK,Skopje,Macedonia,25333,2059794
Serbia,SRB,Beograd,Serbia,77474,7120666
Montenegro,MNE,Podgorica,Montenegro,14026,620029
Kosovo,KOS,Prishtine,Kosovo,10887,1733872
Andorra,AND,Andorra la Vella,Andorra,450,78115
France,F,Paris,Île-de-France,547030,64933400
Spain,E,Madrid,Madrid,504750,46815916
Austria,A,Wien,Wien,83850,8499759


A quick way to count the total number of records in a table is to use the `count()` aggregate function:

In [50]:
%%sql
SELECT count(*) FROM city;

count(*)
3390


This tells us that there are 3111 rows in the `city` table. (More on
aggregation below.)

JupySQL also has a handy "Table Explorer" feature that lets you browse a table interactively inside of Jupyter Notebook—neat!

In [34]:
%sqlcmd explore --table country

## SQL expressions in WHERE clauses

As we saw in one of the above examples, the `WHERE` clause requires an
expression that returns true or false. Rows in the table for which this
expression is true will be included in the result set; rows for which
this expression evaluates to false will be skipped.

The syntax for SQL expressions is, in general, very similar to the
syntax for writing expressions in (e.g.) Python (the analog for the
`WHERE` clause in Python would be the expression following `if` in a
list comprehension). There are a number of operators which take
expressions on either side; these expressions can be either literals
(such as numbers or strings that you type directly into the query) or
column names. If the expression is a column name, then its value is the
contents of that column for the row currently being evaluated.

The supported operators in SQL are, in some cases, slightly different
from their counterparts in Python. Here are some common SQL operators:

| operator     | description                        |
|--------------|------------------------------------|
| `>`          | greater than                       |
| `<`          | less than                          |
| `>=`         | greater than or equal to           |
| `<=`         | less than or equal to              |
| `=`          | equal to (note: `=` and not `==`!) |
| `<>` or `!=` | not equal to                       |

To check to see if a value is `NULL` or missing, use the special
expression `IS NULL` (or, for the converse, `IS NOT NULL`).

> You can also use basic mathematical expressions in SQL, to check if
> (e.g.) the sum of the value in two columns is greater than a
> particular value. [Here’s a
> list](https://www.sqlite.org/lang_expr.html)
> of the expression syntax in SQLite, including supported mathematical operators.

Here’s a quick example of using the `>` operator to find all rows in the
`lake` table with an `area` value of greater than 30000:

In [51]:
%%sql
SELECT name, area, depth FROM lake WHERE area > 30000;

Name,Area,Depth
Ozero Baikal,31492,1637
Dead Sea,41650,378
Caspian Sea,386400,995
Lake Victoria,68870,85
Lake Tanganjika,32893,1470
Great Bear Lake,31792,446
Lake Huron,59600,229
Lake Michigan,57800,281
Lake Superior,82103,405


### More sophisticated WHERE clauses with `AND` and `OR`

You can construct more sophisticated expressions in your `WHERE` clauses
using the `AND` and `OR` operators. These work just like their Python
counterparts: on either side, write an expression. If both expressions
return true, then the entire expression with `AND` returns true. If
either returns true, then the entire expression with `OR` returns true.

As a quick example, let’s find all of the cities in Finland that meet a
particular level of population. The cities in the `city` table are
linked to their country with a country code in the `country` field. We
can determine what that country code is by running a `SELECT` on the
`country` table:

In [52]:
%%sql
SELECT name, code FROM country WHERE name = 'Finland';

Name,Code
Finland,SF


Now we can query the `city` table for Finnish cities:

In [53]:
%%sql
SELECT name, country, population FROM city WHERE country = 'SF';

Name,Country,Population
Mariehamn,SF,10851
Hämeenlinna,SF,67803
Lahti,SF,103396
Tampere,SF,220678
Kuopio,SF,106475
Lappeenranta,SF,72617
Kouvola,SF,85306
Rovaniemi,SF,61244
Mikkeli,SF,54633
Jyväskylä,SF,134862


Let’s say we want to find all Finnish cities with a population of at
least 100000 people. We can write that query like so:

In [54]:
%%sql
SELECT name, country, population
FROM city
WHERE country = 'SF' AND population > 100000;

Name,Country,Population
Lahti,SF,103396
Tampere,SF,220678
Kuopio,SF,106475
Jyväskylä,SF,134862
Oulu,SF,194181
Turku,SF,182154
Helsinki,SF,614535
Espoo,SF,261654


Another example: the `lake` table has a list of lakes, along with (among
other things) their area and depth. Let’s say that we want to find a
list of Earth’s most remarkable lakes, based on those lakes’ depth
and area. To get a list of lakes that are either (a) 500 meters deep or
(b) have a surface area of more than 30000 square meters:

In [55]:
%%sql
SELECT name, area, depth
FROM lake
WHERE depth > 500 OR area > 30000;

Name,Area,Depth
Ozero Baikal,31492,1637
Dead Sea,41650,378
Caspian Sea,386400,995
Issyk-Kul,6236,668
Lake Pinatubo,1.8,600
Lake Lanao,354.6,701
Lake Toba,1103,505
Lake Nyos,1.58,1091
Lake Victoria,68870,85
Lake Tanganjika,32893,1470


## Sums, minimums, maximums and averages

Let’s say you wanted to find the population of the entire world. The `country` table is our best bet for finding this data, having as it does a `population` field:

In [56]:
%%sql
SELECT population FROM country LIMIT 10

Population
2821977
10816286
2059794
7120666
620029
1733872
78115
64933400
46815916
8499759


Presumably, excepting the number of stateless individuals not counted
among the population of a particular country, we could determine the
world’s population by adding up all of these numbers. To save us the
tedium of writing a program to perform this task, SQL provides a
particular kind of syntax to calculate sums for all of the values in a
field. It looks like this:

In [57]:
%%sql
SELECT sum(population) FROM country

sum(population)
7082721908


The new part here is `sum()`, with the field you want summed between the
parentheses. The `sum()` function is one of several so-called
“[aggregate functions](https://www.sqlite.org/lang_aggfunc.html)”
that take the all the values from a field and reduce them down to a
single value. Another such function is `avg()`, which calculates the
arithmetic mean of a column of values:

In [58]:
%%sql
SELECT avg(population) FROM country

avg(population)
29027548.80327869


You can use a `WHERE` clause with these queries to limit which rows are
included in the aggregate. For example, the following query selects all
the MONDIAL cities in Finland:

In [59]:
%%sql
SELECT population FROM city WHERE country = 'SF'

Population
10851
67803
103396
220678
106475
72617
85306
61244
54633
134862


Adding `sum()` around the `population` column yields the sum of just
these values from the table:

In [60]:
%%sql
SELECT sum(population) FROM city WHERE country = 'SF'

sum(population)
2541672


We already covered another aggregate function, `count()`, which simply counts the number of rows. To illustrate this with another example, consider the `encompasses` table, which relates countries to the continents that encompass them. (Browse this table with a `SELECT * FROM continents` to familiarize yourself with the structure.)
To count the number of countries at least partially in Europe:

In [61]:
%%sql
SELECT count(country) FROM encompasses WHERE continent = 'Europe'

count(country)
54


Finally, the `min()` and `max()` aggregate functions return,
respectively, the minimum and maximum values for the given column. To
find the country with the smallest area, we might issue the following
query:

In [62]:
%%sql
SELECT min(area) FROM country

min(area)
0.44


To find the are value for the country with the largest largest area:

In [63]:
%%sql
SELECT max(area) FROM country

max(area)
17075200


SQL lets you select more than one aggregate function in the same query:

In [64]:
%%sql
SELECT min(area), max(area) FROM country

min(area),max(area)
0.44,17075200


### SELECTs with aggregate functions are different

We had been using the `SELECT` statement before as, essentially, a way
to filter and order rows from a table based on their characteristics. So
you may, at this point, notice that `SELECT` statements that include
aggregate functions operate differently from their counterparts without
any aggregate functions. Although the results of the query are returned
in a tabular format, the “rows” and “columns” in the result don’t
correspond to rows and columns in the original table. (E.g., there is no
column in the `country` table called `min` and `max`; these appear only
in the results of a query using those aggregate functions.)

I bring this up because it’s worth pointing out that `SELECT` with
aggregates is a little bit counterintuitive. Personally, I wish that the
syntax made this a bit clearer; everyone would benefit if, in order to
use aggregate functions, you had to use a separate statement (like
`AGGREGATE table CALCULATE min(x)` or something like that, sort of like
[how MongoDB does
it](http://docs.mongodb.org/manual/reference/method/db.collection.aggregate/)).
But that’s not the way SQL works, and so we close our eyes, take a deep
breath, and entrust ourselves to the solutions and abstractions so
carefully invented by the standards-makers of government and industry.

### Aggregating with `GROUP BY`

There’s a particular pattern for using aggregate function that happens
over and over frequently enough that there is a special syntax for it:
grouping. To illustrate, consider the following task: we want to find
the population number for the largest city in each country, using the
data in the `city` table. We already know how to do this for individual
countries, in separate queries; here are two such queries for the US and
Finland:

In [65]:
%%sql
SELECT max(population) FROM city WHERE country = 'USA'

max(population)
8405837


In [66]:
%%sql
SELECT max(population) FROM city WHERE country = 'SF'

max(population)
614535


If we wanted to do this with *every* country present in the `city`
table, we’d have a little programming task on our hands: find all of the
unique countries, iterate through them, issue a query for each, etc.
etc. etc. Because this task is so common, SQL provides a shortcut, which
is the `GROUP BY` clause. If you include a `GROUP BY` clause in your SQL
statement, the aggregate that you specify will be performed *not* on all
of the rows that match the `WHERE` clause, but for all rows having
unique values for the column you specify.

For example, the following query calculates the maximum value for the
`population` column for every unique country:

In [67]:
%%sql
SELECT country, max(population) FROM city GROUP BY country;

Country,max(population)
A,1761738
AFG,2435400
AG,22219
AL,418495
AMSA,None
AND,22256
ANG,6760439
ARM,1066264
ARU,None
AUS,4605992


Some countries have blanks, since apparently there are some countries
whose listed cities all have an empty population field. This query
tells us that, e.g., the most populous city in Nepal has 1003285 people,
that the most populous city in Argentina (code `RA`) has 2768772 people,
etc.

We can clean up the empty rows by using a `WHERE` clause to include as
candidates for aggregation only those cities that have a non-empty
`population` field:

In [69]:
%%sql
SELECT country, max(population)
FROM city
WHERE population IS NOT NULL
GROUP BY country

Country,max(population)
A,1761738
AFG,2435400
AG,22219
AL,418495
AND,22256
ANG,6760439
ARM,1066264
AUS,4605992
AZ,2150800
B,507911


Queries with `GROUP BY` also allow you to use the `ORDER BY` and `LIMIT`
clauses. Here’s an example that sorts the results of the query in
alphabetical order by country, limited to just the first five rows:

In [70]:
%%sql
SELECT country, max(population)
FROM city
WHERE population IS NOT NULL
GROUP BY country
ORDER BY country
LIMIT 5

Country,max(population)
A,1761738
AFG,2435400
AG,22219
AL,418495
AND,22256


To order by the aggregate field, repeat the aggregate expression in the
`ORDER BY` clause:

In [71]:
%%sql
SELECT country, max(population)
FROM city
WHERE population IS NOT NULL
GROUP BY country
ORDER BY max(population) DESC
LIMIT 5

Country,max(population)
CN,22315474
PK,14916456
WAN,13745000
TR,13710512
IND,12442373


If you're curious, the easiest way to get the *name* of the city with the largest population for each country is with a *subquery* in the `WHERE` clause, like so:

In [76]:
%%sql
SELECT country, name
FROM city c1
WHERE population = (
    SELECT max(c2.population)
    FROM city c2
    WHERE c1.country = c2.country
)
LIMIT 12

Country,Name
AL,Tirana
GR,Athina
MK,Skopje
SRB,Beograd
MNE,Podgorica
KOS,Prishtine
AND,Andorra la Vella
F,Paris
E,Madrid
A,Wien


We'll discuss a few more of the mechanics behind queries like this a bit later.

Here’s another example of `GROUP BY`. Consider the `island` table, which
is a list of islands in the world, including their area, height, island
group and type:

In [77]:
%%sql
SELECT name, islands, area, elevation, type FROM island LIMIT 10

Name,Islands,Area,Elevation,Type
Svalbard,Svalbard,39044,1713,None
Greenland,None,2175600,3694,None
Iceland,None,102829,2110,volcanic
Austvågøy,Lofotes,526,1146,None
Streymoy,Faroe Islands,373,789,None
Ireland,British Isles,84421,1041,None
Great Britain,British Isles,219331,1344,None
Shetland Mainland,Shetland Islands,970,449,None
Orkney Mainland,Orkney Islands,492,271,None
South Ronaldsay,Orkney Islands,50,118,None


The `type` field has a few distinct values:

In [78]:
%%sql
SELECT DISTINCT(type) FROM island

Type
None
volcanic
lime
coral
atoll


So, let’s find the *average area* of islands belonging to each island
type:

In [79]:
%%sql
SELECT type, avg(area) FROM island GROUP BY type

Type,avg(area)
None,35378.35155555556
atoll,57.137
coral,212.38333333333333
lime,298.6
volcanic,8964.798003809523


Volcanic islands are awful big, aren’t they?

> EXERCISE: Find the island group (i.e., the `islands` field in the
> `island` table) with the greatest average height.

### Filter aggregations with `HAVING`

We saw above that the `WHERE` clause can be used to restrict which rows
are used when calculating an aggregate. But what if we want to restrict
which rows are present *in the response to the aggregate query* itself?
It’s not difficult, but it does require a discussion of a previously
undiscussed clause: `HAVING`.

To illustrate the problem, consider the `river` table. This table has a
list of rivers, which includes the river’s name and its outlet, which is
either a sea, a lake, or another river (or possibly none of these, in
the case of rivers in endorheic basins). Here’s what the table looks
like:

In [80]:
%%sql
SELECT name, river, lake, sea FROM river LIMIT 10;

Name,River,Lake,Sea
Thjorsa,None,None,Atlantic Ocean
Jökulsa a Fjöllum,None,None,Greenland Sea
Thames,None,None,North Sea
Severn,None,None,Atlantic Ocean
Trent,None,None,North Sea
Clyde,None,None,Irish Sea
Leven,Clyde,None,None
Ness,None,None,North Sea
Bann,None,None,Atlantic Ocean
Shannon,None,None,Atlantic Ocean


These results show that, e.g., the [Thjorsa
river](https://en.wikipedia.org/wiki/%C3%9Ej%C3%B3rs%C3%A1) empties into
the Atlantic Ocean, while the [Klaraelv
river](https://en.wikipedia.org/wiki/Klar%C3%A4lven) empties into a lake
named “Vaenern.”

Let’s say we’re interested in knowing exactly how many rivers empty into
all of the known seas. We could find this out by issuing a query that
counts the rivers, grouped by the name of the sea:

In [81]:
%%sql
SELECT sea, count(name) FROM river GROUP BY sea

Sea,count(name)
None,267
Andaman Sea,2
Arabian Sea,2
Arctic Ocean,3
Atlantic Ocean,37
Baltic Sea,14
Barents Sea,2
Bering Sea,1
Black Sea,3
Caribbean Sea,8


Pretty good so far. But what if we wanted to get this list of results, but *exclude* the rows that have a count of five or less. Is this possible? If so, how?

The first way you might attempt to solve this would be with the `WHERE` clause. That’s what `WHERE` is for, after all, right? However, tt turns out that the `WHERE` clause can only be used to filter rows *before* the aggregation operation happens—not afterward. To filter rows in the aggregation, there’s a different clause, (confusingly, IMO) called `HAVING`. The `HAVING` clause works like this:

In [84]:
%%sql
SELECT sea, count(name) FROM river GROUP BY sea HAVING count(name) > 5

Sea,count(name)
None,267
Atlantic Ocean,37
Baltic Sea,14
Caribbean Sea,8
Mediterranean Sea,20
North Sea,7
Pacific Ocean,13
South China Sea,8


The `HAVING` clause looks just like a `WHERE` clause, except that it can
refer only to fields that are present in the aggregation (in this case,
`count(name)`). As you can see, by including the `HAVING` clause in this
query, we’ve excluded results where the aggregation function didn’t meet
a certain criterion. Perfect!

Let’s do another example. The `religion` table is a list of records that
relate religions to countries. Each country has several records in the
table, and each record indicates the percentage of the population that
adherents to the given religion make up in the country. So, for example,
this query:

In [85]:
%%sql
SELECT * FROM religion LIMIT 10

Country,Name,Percentage
AL,Muslim,70
AL,Roman Catholic,10
AL,Christian Orthodox,20
GR,Christian Orthodox,98
GR,Muslim,1.3
MK,Christian Orthodox,64.7
MK,Muslim,33.3
MK,Roman Catholic,0.2
SRB,Christian Orthodox,85
SRB,Muslim,3.2


… shows that in Albania (code `AL`), 70 percent of the country is
Muslim, 10 percent is Roman Catholic, and 20 percent is Christian
Orthodox.

We’ll use this table to find religions that are present in the fewest
countries.

In [86]:
%%sql
SELECT name, count(country) FROM religion GROUP BY name

Name,count(country)
Adventist,1
African Methodist Episcopal,1
Anglican,17
Armenian Apostolic,1
Bahai,2
Baptist,4
Buddhist,27
Cao Dai,1
Chondogyo,1
Christian,59


Let’s add a `HAVING` clause so that we see *only* the religions that are present
in a single country:

In [87]:
%%sql
SELECT name, count(country)
FROM religion
GROUP BY name
HAVING count(country) = 1

Name,count(country)
Adventist,1
African Methodist Episcopal,1
Armenian Apostolic,1
Cao Dai,1
Chondogyo,1
Christian Congregationalist,1
Church Christ,1
Church Tuvalu,1
Confucianism,1
Congregational Christian,1


(Obviously, there are adherents of these religions in more than one
country! Presumably, the `religion` table only has records if the number
of adherents is large enough that it constitutes a percentage of the
general population above a certain threshold.)

Let’s make our query a bit more specific and find the religions that
only occur in one country and where that religion’s percentage share in
the country is less than 5%. We can do this by filtering the rows first
with `WHERE`, like so:

In [88]:
%%sql
SELECT name, count(country)
FROM religion
WHERE percentage < 5
GROUP BY name
HAVING count(country) = 1

Name,count(country)
Adventist,1
Cao Dai,1
Chondogyo,1
Church Christ,1
Confucianism,1
Druze,1
Hoa Hao,1
Jains,1
Mayan,1
New Apostolic,1


The tricky part in this query is the *combination* of `WHERE` and
`HAVING`. The `WHERE` clause tells SQL to exclude any rows where the
percentage is less than 5 *before* any aggregation happens. The `HAVING`
clause tells SQL to exclude any rows *after* the aggregation where the
result of `count(country)` is not equal to 1.

## Joins

In this section, we’re going to discuss one of the things that makes SQL
truly powerful: the ability to create queries that “join” tables
together.

To illustrate, let’s tackle one particular task. So far, We’ve been
running up against a problem pretty consistently with the MONDIAL
database, which is this: the *names* of countries aren’t stored in most
of these tables—just their “code.” When looking at the `city` table, for
instance:

In [89]:
%%sql
SELECT name, country FROM city ORDER BY name LIMIT 10

Name,Country
's-Hertogenbosch,NL
??ëlkovo,R
?achty,R
?anl?urfa,TR
?eleznodoro?nyj,R
?erkessk,R
?eské Bud?jovice,CZ
?ilina,SK
?negöl,TR
?ngel's,R


Unless we happen to have already memorized these codes, we have to look
them up one-by-one in the `country` table to find out what they mean:

In [90]:
%%sql
SELECT name FROM country WHERE code = 'WAN'

Name
Nigeria


That’s sort of inconvenient, and it seems like computers should be able
to help with this problem. Isn’t there a way to write a query so that
each row returned from the `city` table automatically gets matched up
with the `country` that has the corresponding code?

In fact, there is, and it’s called `JOIN`. We’ll continue with the
`city`/`country` analogy in a second, but it’s a bit easier to
demonstrate how `JOIN` works with some smaller, toy tables first. Recall
from Part One our tiny database for a news organization, which consists
of a table for writers:

| name                 | title        | start_year |
|----------------------|--------------|------------|
| Gabriella McCullough | reporter     | 2009       |
| Steven Kennedy       | drama critic | 2012       |
| Jalen Shaara         | columnist    | 2002       |

… and then a table of articles that those writers are responsible for:

| author               | title                                         | published_date |
|----------------------|-----------------------------------------------|----------------|
| Gabriella McCullough | Man, opossum reach garbage accord             | 2015-07-01     |
| Steven Kennedy       | “The Deceit of Apricot” opens to rave reviews | 2015-07-15     |
| Jalen Shaara         | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16     |
| Gabriella McCullough | Traffic signals restored on Tunguska Ave      | 2015-07-01     |

Let’s say we wanted to produce a *new* table, which consists of a list
of article titles and dates, along with the name of the author, the
author’s title, and the author’s start year. In other words, we want
information from *both* tables, and we want to automatically *align*
that data so that we end up with the correct title and start year for
each author. Basically, what we want is this:

| article.author       | article.title                                 | article.published_date | author.title | author.start_year |
|----------------------|-----------------------------------------------|------------------------|--------------|-------------------|
| Gabriella McCullough | Man, opossum reach garbage accord             | 2015-07-01             | reporter     | 2009              |
| Steven Kennedy       | “The Deceit of Apricot” opens to rave reviews | 2015-07-15             | drama critic | 2012              |
| Jalen Shaara         | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16             | columnist    | 2002              |
| Gabriella McCullough | Traffic signals restored on Tunguska Ave      | 2015-07-01             | reporter     | 2002              |

Essentially, what we’ve done is taken our “articles” table, and then
reshuffled the “authors” table and glued it on to the right-hand side,
making one big monster table that joins the two together. This is what
is meant by a “join” in relational database parlance.

We’re going to solve our country name problem using a join as well, and
in the process, explain the syntax for how joins work in SQL.

### Join in SQL

The syntax of a `JOIN` looks like this:

    SELECT fields
    FROM left_table
    JOIN right_table ON left_field = right_field

… where `fields` is the list of fields that you want, `left_table` is
the table you want to leave alone, and `right_table` is the table you
want to re-arrange and tack on to the left table. The `left_field` and
`right_field` values determine how the joined table will be aligned: the
data from `right_table` will be joined to the

A join consists of two tables, and a field in each table that links the
two together. In the example above, the “link” between the two tables is
that the name of the author in the articles table needs to match the
name of the author in the writers table. For the purposes of naming the
countries that each city is in, the two tables we want to join are
`city` and `country`, and the “link” between them is the country
code—which is in the `country` field of the `city` table, and the `code`
field of the `country` table. Here’s what the query looks like:

In [91]:
%%sql
SELECT city.name, city.population, country.code, country.name
FROM city JOIN country ON city.country = country.code
LIMIT 10

Name,Population,Code,Name_1
Tirana,418495,AL,Albania
Shkodër,77075,AL,Albania
Durrës,113249,AL,Albania
Vlorë,79513,AL,Albania
Elbasan,78703,AL,Albania
Korçë,51152,AL,Albania
Komotini,None,GR,Greece
Kavala,58790,GR,Greece
Athina,664046,GR,Greece
Peiraias,163688,GR,Greece


There’s a *lot* going on in this query, and one or two new things other
than the join. So let’s take it line by line. Let’s start with the
middle line:

    FROM city JOIN country ON city.country = country.code

This line says to select from the `city` table (this is the left table
of our join) and join it to the `country` table (the right table). The
`ON` clause tells PostgreSQL how to re-arrange the right-side table. It
says, in effect: for every row in the `city` table, find the row in the
`country` table where the country’s `code` field (`country.code`)
matches the city’s `country` field (`city.country`).

The dot between the name of the table and the field is something we
haven’t discussed yet: when you’re naming a field, the dot syntax allows
you to specify *which table* that field is found in. (This is important
when the two tables you’re joining have fields with the same name—you
need to be able to disambiguate.) We see the dot again in the first line
of the query:

    SELECT city.name, city.population, country.code, country.name

This line specifies *which* fields we want to see. Because we’re joining
two tables, we need to be explicit about which table the field we want
originates from. In this query, we’re getting the `name` field from the
`city` table, the `population` field from the `country` table, the
`code` field from the `country` table, and the `name` field from the
`country` table.

> NOTE: To see what the entire joined table looks like, without any
> fields selected, try the following query:
> `SELECT * FROM city JOIN country ON city.country = country.code LIMIT 10;`

Finally, the `LIMIT 10` line works just like it does with other `SELECT`
statements: it just limits the results to the given number of lines.

### Combining `JOIN` with `WHERE` and aggregation

Once you’ve created a joined table with a `JOIN` clause, you can operate
on it just like any other table—restricting selections with `WHERE` and
performing aggregates with `GROUP BY`. Let’s do another example from
joining the `city` and `country` tables. Here’s a query that finds every
city with a population over one million people that is found in a
country with fewer than five million people:

In [92]:
%%sql
SELECT city.name, city.population, country.name,
country.population
FROM city JOIN country ON city.country = country.code
WHERE city.population > 1000000 AND country.population < 5000000

Name,Population,Name_1,Population_1
Yerevan,1066264,Armenia,3026879
Tbilisi,1073345,Georgia,4483800
Bayr?t,1100000,Lebanon,4341092
Montevideo,1318755,Uruguay,3286314
Brazzaville,1408150,Congo,4001831
Monrovia,1010970,Liberia,3957990


A great thing about table joins is that we can use `WHERE` to establish
criteria for fields in either the left or right table. If you’re having
trouble picturing how this query works, try running it without one of
the expressions in the `WHERE` clause (i.e., leave out
`city.population > 1000000` or `country.population < 5000000`).

The following example combines nearly all of the concepts we’ve
discussed so far. It finds the sum of the population of cities in the
`city` table for all countries, and then displays those countries having
at least 20 million individuals living in cities.

In [93]:
%%sql
SELECT country.name, sum(city.population)
FROM city JOIN country ON city.country = country.code
WHERE city.population IS NOT NULL
GROUP BY country.name
HAVING sum(city.population) > 20000000
ORDER BY sum(city.population) DESC

Name,sum(city.population)
China,326058186
India,129752758
Brazil,93687185
United States,81882315
Russia,72000673
Pakistan,50920843
Nigeria,49410900
Japan,48415803
Mexico,47137170
Indonesia,46627466


The tricky part here is the `GROUP BY` clause, which is grouping by a
value in the right table of the join (`country.name`).

## Joining with many-to-many relationships

So far, we’ve been exercising our table join chops on the `city` and
`country` tables. These two tables have a *many-to-one* relationship:
one country can contain many cities, but a city can only be in one
country. It’s easy to write a `JOIN` for a one-to-many relation, since
you know that the right-side table will always have, at most, one
matching record.

But the MONDIAL database (along with many other relational database
schemas) has entities that exist in a many-to-many relationship. For
example, a river can flow through multiple countries, and one country
can have multiple rivers flowing through it. Representing many-to-many
relationships in SQL is a bit tricky, and as a consequence, writing
`JOIN`s for many-to-many relationships is tricky as well.

The conventional way to model a many-to-many relationship in a
relational database is with a [junction
table](https://en.wikipedia.org/wiki/Junction_table) (sometimes also
called a “linking table”). A junction table has rows for every instance
of relationship between two tables, using a unique key to refer to the
rows in those tables.

### Many writers, many articles

To illustrate, let’s return to our news organization database. We have a
table for writers:

| name                 | title        | start_year |
|----------------------|--------------|------------|
| Gabriella McCullough | reporter     | 2009       |
| Steven Kennedy       | drama critic | 2012       |
| Jalen Shaara         | columnist    | 2002       |

… and then a table of articles that those writers are responsible for:

| author               | title                                         | published_date |
|----------------------|-----------------------------------------------|----------------|
| Gabriella McCullough | Man, opossum reach garbage accord             | 2015-07-01     |
| Steven Kennedy       | “The Deceit of Apricot” opens to rave reviews | 2015-07-15     |
| Jalen Shaara         | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16     |
| Gabriella McCullough | Traffic signals restored on Tunguska Ave      | 2015-07-01     |

This schema represents a simple many-to-one relationship: one writer can
write many articles, and every article has exactly one writer. But let’s
say that one day, in our news organization, Gabriella McCullough and
Steven Kennedy *collaborate* on an article. How do we represent this in
the database?

One option, of course, would simply be to store both of the names in the
`author` field:

| author                                  | title                                      | published_date |
|-----------------------------------------|--------------------------------------------|----------------|
| Gabriella McCullough and Steven Kennedy | Theater Chairs Uncomfortable, Experts Warn | 2015-07-28     |

There’s a problem with this solution, however, which is that now a query
on our database that looks like this:

    SELECT count(*) FROM articles WHERE author = 'Gabriella McCullough';

… no longer functions properly, because it won’t include the article
where Gabriella collaborated with Steven. Likewise, it would be
difficult to construct a `JOIN` (like we did in the previous section)
that would tell us the title of all of the authors involved in writing
the story. (We’d have to parse out the names first in order to use them
in the query, which is a hassle.)

The issue is that we’ve discovered that our initial modeling of the data
structure was wrong. There isn’t a many-to-one relationship between
articles and writers; instead, there’s a many-to-many relationship: a
single writer can write multiple stories, and any given story can be
authored by more than one writer.

To represent this relationship, we need to slightly restructure our
database. First, we’ll remove the `author` field from the `articles`
table and add a new field, `article_id`, which is a unique integer
assigned to each article:

| article_id | title                                         | published_date |
|------------|-----------------------------------------------|----------------|
| 1          | Man, opossum reach garbage accord             | 2015-07-01     |
| 2          | “The Deceit of Apricot” opens to rave reviews | 2015-07-15     |
| 3          | What’s the Big Data? Why I’m a data skeptic   | 2015-07-16     |
| 4          | Traffic signals restored on Tunguska Ave      | 2015-07-01     |
| 5          | Theater Chairs Uncomfortable, Experts Warn    | 2015-07-28     |

… and then create a new table, which relates author names to the
articles that they’ve written. We’ll store one row for each instance of
a relationship between an author and an article and call it
`author_article`:

| author               | article_id |
|----------------------|------------|
| Gabriella McCullough | 1          |
| Gabriella McCullough | 4          |
| Gabriella McCullough | 5          |
| Steven Kennedy       | 2          |
| Steven Kennedy       | 5          |
| Jalen Shaara         | 3          |

This junction table tells us that Gabriella has a byline on articles 1,
4, and 5; Steven has a byline on articles 2 and 5; and Jalen has a
byline on article 3. For any article, we could get a list of its authors
like so:

    SELECT author FROM author_article WHERE article_id = 5;

Getting a list of titles on which a writer has a byline is slightly more
complicated, and involves a join:

    SELECT article.title
    FROM author_article JOIN article
        ON article.article_id = author_article.article_id
    WHERE author_article.author = 'Gabriella McCullough';

### Rivers and countries

Let’s return to the MONDIAL database for an example with real data. As
mentioned above, rivers and countries are in a many-to-many
relationship. The MONDIAL database has a table for countries, and a
table for rivers, and a table called `geo_river` that is the junction
table for their many-to-many relationship. Here’s what the table looks
like:

In [96]:
%%sql
SELECT river, country, province FROM geo_river ORDER BY river LIMIT 20;

River,Country,Province
Aare,CH,Aargau
Aare,CH,Bern
Aare,CH,Solothurn
Acre,BOL,Pando
Acre,BR,Acre
Acre,BR,Amazonas
Acre,PE,Madre de Dios
Adda,I,Lombardia
Agus River,RP,Northern Mindanao
Agus River,RP,Muslim Mindanao


This result shows us that, e.g., the Amazon river flows through several
different countries. (The `geo_river` table also gives us information on
which *provinces* a river flows through, so we see that the Allegheny
wends its way through both New York and Pennsylvania.)

All well and good so far. Let’s exploit the many-to-many relationship to
get information about particular countries and rivers. To find all of
the rivers that flow through Finland:

In [97]:
%%sql
SELECT river, province FROM geo_river WHERE country = 'SF';

River,Province
Paatsjoki,Lappia
Torneälv,Lappia
Ounasjoki,Lappia
Kemijoki,Lappia
Oulujoki,Pohjois-Pohjanmaa
Oulujoki,Kainuu
Kymijoki,Päijät-Häme
Kymijoki,Kymenlaakso
Kokemäenjoki,Pirkanmaa
Kokemäenjoki,Satakunta


A list of all the countries and provinces that the Rhine runs through:

In [98]:
%%sql
SELECT river, country, province FROM geo_river WHERE river = 'Rhein'

River,Country,Province
Rhein,F,Alsace
Rhein,A,Vorarlberg
Rhein,D,Baden-Württemberg
Rhein,D,Hessen
Rhein,D,Nordrhein-Westfalen
Rhein,D,Rheinland-Pfalz
Rhein,FL,Liechtenstein
Rhein,CH,Aargau
Rhein,CH,Basel-Landschaft
Rhein,CH,Basel-Stadt


This is a bit annoying, since we’re seeing the country codes again
instead of the country names. In order to get the country name, we need
to `JOIN` the junction table with the `country` table to get the country
name out.

In [99]:
%%sql
SELECT geo_river.river, country.name, geo_river.province
FROM geo_river JOIN country ON geo_river.country = country.code
WHERE river = 'Rhein'

River,Name,Province
Rhein,France,Alsace
Rhein,Austria,Vorarlberg
Rhein,Germany,Baden-Württemberg
Rhein,Germany,Hessen
Rhein,Germany,Nordrhein-Westfalen
Rhein,Germany,Rheinland-Pfalz
Rhein,Liechtenstein,Liechtenstein
Rhein,Switzerland,Aargau
Rhein,Switzerland,Basel-Landschaft
Rhein,Switzerland,Basel-Stadt


Let’s dig a bit deeper and select rivers based on their characteristics.
We already know how to get a list of names of rivers whose length is
greater than a certain number, say 4000:

In [102]:
%%sql
SELECT name FROM river WHERE length > 4000

Name
Irtysch
Jenissej
Lena
Hwangho
Yangtze
Mekong
Missouri
Niger
Zaire


But let’s say that we want to know the names of the countries in which
these rivers at least partially lie. In order to find this, we’re going
to `JOIN` this statement to the `geo_river` table. Here’s the query:

In [103]:
%%sql
SELECT river.name, geo_river.country, geo_river.province
FROM river JOIN geo_river ON river.name = geo_river.river
WHERE river.length > 4000

Name,Country,Province
Irtysch,R,Omskaya
Irtysch,R,Tyumenskaya
Irtysch,R,Khanty Mansi ao
Irtysch,CN,Xinjiang
Irtysch,KAZ,Pavlodar
Irtysch,KAZ,East-Kazakhstan
Jenissej,R,Tyva
Jenissej,R,Khakasiya
Jenissej,R,Krasnoyarsk
Lena,R,Irkutskaya


This join is interesting, since in the act of joining, we actually
introduced *more rows* into the search results. That’s what happens when
the right table has more than one row that matches the condition in the
`ON` clause.

Of course, we still have that pesky country code! To get rid of it, we need
to join *twice*: once on `geo_river` and again on `country`:

In [105]:
%%sql
SELECT river.name, country.name, geo_river.province
FROM river
JOIN geo_river ON river.name = geo_river.river
JOIN country ON country.code = geo_river.country
WHERE river.length > 4000

Name,Name_1,Province
Irtysch,Russia,Omskaya
Irtysch,Russia,Tyumenskaya
Irtysch,Russia,Khanty Mansi ao
Irtysch,China,Xinjiang
Irtysch,Kazakhstan,Pavlodar
Irtysch,Kazakhstan,East-Kazakhstan
Jenissej,Russia,Tyva
Jenissej,Russia,Khakasiya
Jenissej,Russia,Krasnoyarsk
Lena,Russia,Irkutskaya


This query essentially takes the table resulting from the first join and
uses it as the left table in a *second* join. Tricky! But powerful.

### Aggregates with many-to-many relations

You can use aggregates with junction tables fairly easily. For example,
here’s a query that gets the total number of provinces that each river
passes through:

In [106]:
%%sql
SELECT river, count(province)
FROM geo_river GROUP BY river
ORDER BY count(province) DESC
LIMIT 10

River,count(province)
Donau,33
Niger,20
Rhein,18
Euphrat,17
Tigris,16
Zaire,14
Zambezi,13
Volga,13
Elbe,12
Dnepr,12


And, the other way, the total number of rivers that pass through each
province:

In [107]:
%%sql
SELECT province, count(river)
FROM geo_river
GROUP BY province
ORDER BY count(river) DESC
LIMIT 10

Province,count(river)
Amazonas,14
Bayern,13
Xizang,12
Baden-Württemberg,8
California,7
Nepal,7
Ontario,7
Serbia,7
Central African Republic,6
Colorado,6


### DISTINCT and COUNT(DISTINCT …)

We discussed the `DISTINCT` keyword above when exploring the `island`
table’s `type` field. Putting the word `DISTINCT` before a field name
causes the query to return *only the unique values* for the selected
field. `DISTINCT` is an easy way to remove duplicates from results,
especially in the cases where you’re querying only on a single field.

As an example, the `ethnicgroup` table has a record that relates
countries, ethnic groups, and the percentage of the country’s population
that a particular ethnic group makes up. If you simply wanted a list of
*all the ethnic groups in the world*, the one way to structure the query
would be like this:

In [121]:
%%sql
SELECT name FROM ethnicgroup ORDER BY name LIMIT 10

Name
Acholi
Afar
Afar
African
African
African
African
African
African
African


This, of course, is showing us not the list of ethnic groups but the
list of ethnic groups as they relate to countries and percentages, which
means we’ll see the same group multiple times. The easiest way to get
the list of *unique* ethnic groups is to simply put the word `DISTINCT`
before the field in the query:

In [122]:
%%sql
SELECT DISTINCT name FROM ethnicgroup ORDER BY name LIMIT 10

Name
Acholi
Afar
African
African descent
African-white-Indian
Afro-Asian
Afro-Chinese
Afro-East Indian
Afro-European
Albanian


The `DISTINCT` keyword can be used in combination with `COUNT`
aggregation to count the number of times a particular value appears in a
`COUNT` aggregate. As an example, consider the query above where we
counted the number of provinces that a particular river runs through:

In [123]:
%%sql
SELECT river, count(province)
FROM geo_river GROUP BY river
ORDER BY count(province) DESC
LIMIT 10

River,count(province)
Donau,33
Niger,20
Rhein,18
Euphrat,17
Tigris,16
Zaire,14
Zambezi,13
Volga,13
Elbe,12
Dnepr,12


You might think that you could turn this into a query that finds the
number of *countries* that a river passes through simply by switching
the word `province` to `country`:

In [124]:
%%sql
SELECT river, count(country)
FROM geo_river GROUP BY river
ORDER BY count(country) DESC
LIMIT 10

River,count(country)
Donau,33
Niger,20
Rhein,18
Euphrat,17
Tigris,16
Zaire,14
Zambezi,13
Volga,13
Elbe,12
Dnepr,12


But it returns the same results! The reason is that the linking table
has one entry per *province*, and that row includes the country, so
counting the two has essentially the same result. The easy way to get
around this difficulty is by putting the `DISTINCT` keyword inside the
parentheses, which is a special syntax that allows you to include only
the unique values for a particular field inside of the `COUNT`
aggregation. Observe!

In [125]:
%%sql
SELECT river, count(DISTINCT country)
FROM geo_river GROUP BY river
ORDER BY count(DISTINCT country) DESC
LIMIT 10

River,count(DISTINCT country)
Donau,10
Zambezi,6
Rhein,6
Mekong,6
Jordan,5
Drau,5
Senegal,4
Save,4
Rio Putumayo,4
Niger,4


Magic!

## Aliases

To save time and typing, the designers of SQL introduced the concept of
the *alias*. An alias is a way to give a table or column a shorter name
inside of a query, so you have less typing to do. You can also use an
alias to change the name of a column that appears in the output for a
query, which can be especially valuable when working with aggregations.

### Aliasing table names

As an example of where an alias might be useful, consider the query we
wrote earlier that finds cities with large populations in countries with
small populations:

    SELECT city.name, city.population, country.name, country.population
    FROM city JOIN country ON city.country = country.code
    WHERE city.population > 1000000 AND country.population < 5000000;

It seems verbose to write out `city` and `country` so many times. And
whenever something “seems verbose” you can count on programmers having
come up with a way to make it shorter (and maybe a bit more confusing).
So, what you can do is put an `AS` clause after the name of the table in
the `FROM`. Elsewhere in the query, you can now use whatever string you
typed after `AS` to refer to the table, as a kind of shorthand. Here’s
the same query using `AS`:

In [126]:
%%sql
SELECT ci.name, ci.population, co.name, co.population
FROM city AS ci JOIN country AS co ON ci.country = co.code
WHERE ci.population > 1000000 AND co.population < 5000000

Name,Population,Name_1,Population_1
Yerevan,1066264,Armenia,3026879
Tbilisi,1073345,Georgia,4483800
Bayr?t,1100000,Lebanon,4341092
Montevideo,1318755,Uruguay,3286314
Brazzaville,1408150,Congo,4001831
Monrovia,1010970,Liberia,3957990


You get to pick the text of your alias names, but you can't use a [SQLite keyword](https://www.sqlite.org/lang_keywords.html). In general you’ll see a lot of single- and double-character aliases, often using the first letter (or couple of letters) from the name of the table being referenced.

If the `AS` keyword is *still* too verbose for you, you can omit it altogether, leaving just the alias after the table name:

In [127]:
%%sql
SELECT ci.name, ci.population, co.name, co.population
FROM city ci JOIN country co ON ci.country = co.code
WHERE ci.population > 1000000 AND co.population < 5000000

Name,Population,Name_1,Population_1
Yerevan,1066264,Armenia,3026879
Tbilisi,1073345,Georgia,4483800
Bayr?t,1100000,Lebanon,4341092
Montevideo,1318755,Uruguay,3286314
Brazzaville,1408150,Congo,4001831
Monrovia,1010970,Liberia,3957990


### Aliasing column names

You can specify aliases for column names as well, using the same syntax.
A column name alias simply changes the text that appears in the column
header for a given field in the query.

In [128]:
%%sql
SELECT name AS moniker, population AS resident_count
FROM city
LIMIT 5

moniker,resident_count
Tirana,418495
Shkodër,77075
Durrës,113249
Vlorë,79513
Elbasan,78703


The functionality is slightly more useful when working with aggregate
functions. To demonstrate, let’s return to the following query, which
lists countries along with the population value for the most populous
city in the country:

In [129]:
%%sql
SELECT country, max(population)
FROM city
WHERE population IS NOT NULL
GROUP BY country
ORDER BY max(population) DESC
LIMIT 5

Country,max(population)
CN,22315474
PK,14916456
WAN,13745000
TR,13710512
IND,12442373


This query is a bit awkward because we have to write the aggregate
function twice: once in the column list and again in the `ORDER BY`
clause. As an alternative, you can give the aggregate a column alias,
and use that alias in subsequent clauses:

In [130]:
%%sql
SELECT country, max(population) as most_people
FROM city
WHERE population IS NOT NULL
GROUP BY country
ORDER BY most_people DESC
LIMIT 5

Country,most_people
CN,22315474
PK,14916456
WAN,13745000
TR,13710512
IND,12442373


     country | most_people 
    ---------+-------------
     CN      |    22315474
     TR      |    13710512
     IND     |    12442373
     R       |    11979529
     BR      |    11152344
    (5 rows)

As a perk, using a column alias for the aggregate gives the column a more descriptive name in the output (without the alias, PostgreSQL uses the name of the aggregate function as the column name).

## Integrating JupySQL, Python and Pandas

I didn't have time to write these up, but here's some information on how you can use SQL results as Pandas dataframes, and incorporate Python expressions and variables into SQL queries:

* [Store the result of a query in a Python variable](https://jupysql.ploomber.io/en/latest/intro.html#assignment)
* [Convert the result of a query in a Pandas dataframe](https://jupysql.ploomber.io/en/latest/integrations/pandas.html#convert-to-pandas-dataframe)
* [Use Python variables in SQL queries](https://jupysql.ploomber.io/en/latest/user-guide/template.html#parametrization-via-variable)
* [Store Pandas dataframes as SQL tables](https://jupysql.ploomber.io/en/latest/integrations/pandas.html#uploading-a-dataframe-to-the-database)

## Further reading

- The [Python DB API](https://philvarner.github.io/pages/novice-python3-db-api.html) provides a lower-level interface for interacting with databases (which you can use in standalone Python programs, not just Jupyter Notebooks). Most Python database libraries conform to this specification (including [sqlite3](https://docs.python.org/3/library/sqlite3.html), which is included in the standard library).
- Our tutorial only covered *querying* databases, not designing them. Relational database design is a fine art in and of itself. You can find a thorough but accessible introduction to the topic in [A Practical Introduction to Databases](https://runestone.academy/ns/books/published/practical_db/index.html).
- [A Gentle Introduction to SQL](https://a-gentle-introduction-to-sql.readthedocs.io/en/latest/) is a quality, creative commons-licensed SQL tutorial that covers some of the same material as this notebook, using a different tool (desktop GUI software called [DB Browser for SQLite](https://sqlitebrowser.org/)).
- [SQL Teaching](https://www.sqlteaching.com/) has a series of online, interactive tutorials about making queries.
- If you’re looking to buy a book, I like O'Reilly's [Head First SQL](http://shop.oreilly.com/product/9780596526849.do) (NYU students should be able to [access this for free through the NYU library](https://bobcat.library.nyu.edu/permalink/f/1c17uag/nyu_aleph007031163).)
- Test your skills with the [SQL Murder Mystery](https://github.com/NUKnightLab/sql-mysteries).